In [3]:
import numpy as np
from numpy.linalg import inv
import pandas as pd

In [30]:
def read_input_file(filepath): # read the input file
    with open(filepath) as f:
        data = f.readlines()
    
    line_d = r_sing_line(data)
    node_n = 0
    
    # create node data
    N = [] 
    while int(line_d[0]) > node_n:
        
        node_n = int(line_d[0])
        N.append(line_d)
        
        line_d = r_sing_line(data)
        
    n_node = node_n
    n_elem = int((np.sqrt(n_node)-1)**2)
    N = np.array(N)
    
    
    # create element data
    E = np.zeros((n_elem, 5))
    for i in range(n_elem):
        
        E[i] = line_d
        line_d = r_sing_line(data)
    
    
    # create constraint data
    C = []
    if int(line_d[1]) != 0:
        C.append(line_d)
        
    while data != []:
        line_d = r_sing_line(data)
        if int(line_d[1]) != 0:
            C.append(line_d)
    
    C = np.array(C)
        
    return n_node, n_elem, N, E, C

def r_sing_line(data):
    while (data[0][0]=='\n') or (data[0][0]=='%'):
        data.pop(0)
    line_d = np.fromstring(data.pop(0), dtype=float, sep=' ')
    
    return line_d


def local_compute(x, y, nint):
    # input x and y coordinates of elements, e.g. x = [0, 0.5, 0.5, 0], y = [0, 0, 0.5, 0.5]
    psiint = np.zeros((nint, ))
    etaint = np.zeros((nint, ))
    weights = np.zeros((nint, ))
    
    if  nint == 4:
        psiint = np.array([-1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3), -1/np.sqrt(3)])
        etaint = np.array([-1/np.sqrt(3), -1/np.sqrt(3), 1/np.sqrt(3), 1/np.sqrt(3)])
        weights = np.array([1, 1, 1, 1])
        
    elif nint == 9:
        psiint = np.array([-np.sqrt(3/5), 0, np.sqrt(3/5), -np.sqrt(3/5), 0, np.sqrt(3/5), 
                          -np.sqrt(3/5), 0, np.sqrt(3/5)])
        etaint = np.array([-np.sqrt(3/5), -np.sqrt(3/5), -np.sqrt(3/5), 0,0,0,
                          np.sqrt(3/5), np.sqrt(3/5), np.sqrt(3/5)])
        weights = np.array([25/81, 40/81, 25/81, 40/81, 64/81, 40/81, 25/81, 40/81, 25/81])
        
    N = np.zeros((4, nint))
    Npsi = np.zeros((4, nint))
    Neta = np.zeros((4, nint))
    Nx = np.zeros((4, nint))
    Ny = np.zeros((4, nint))
    
    for l in range(nint):
        wl = weights[l]
        psil = psiint[l]
        etal = etaint[l]
        N[0, l] = 0.25*(1-psil)*(1-etal)
        N[1, l] = 0.25*(1+psil)*(1-etal)
        N[2, l] = 0.25*(1+psil)*(1+etal)
        N[3, l] = 0.25*(1-psil)*(1+etal)
        
        Npsi[0,l] = -0.25*(1-etal)
        Npsi[1,l] = 0.25*(1-etal)
        Npsi[2,l] = 0.25*(1+etal)
        Npsi[3,l] = -0.25*(1+etal)
        
        Neta[0,l] = -0.25*(1-psil)
        Neta[1,l] = -0.25*(1+psil)
        Neta[2,l] = 0.25*(1+psil)
        Neta[3,l] = 0.25*(1-psil)
        
    xpsi = np.zeros((nint, ))
    xeta = np.zeros((nint, ))
    ypsi = np.zeros((nint, ))
    yeta = np.zeros((nint, ))
    jaco = np.zeros((nint, ))
    psix = np.zeros((nint, ))
    psiy = np.zeros((nint, ))
    etax = np.zeros((nint, ))
    etay = np.zeros((nint, ))
    Nx = np.zeros((4, nint))
    Ny = np.zeros((4, nint))
    
    for l in range(nint):
        for a in range(4):
            xpsi[l] = xpsi[l]+Npsi[a,l]*x[a]
            xeta[l] = xeta[l]+Neta[a,l]*x[a]
            ypsi[l] = ypsi[l]+Npsi[a,l]*y[a]
            yeta[l] = yeta[l]+Neta[a,l]*y[a]
            
        jaco[l] = xpsi[l]*yeta[l] - xeta[l]*ypsi[l]
        psix[l] = yeta[l]/jaco[l]
        psiy[l] = -xeta[l]/jaco[l]
        etax[l] = -ypsi[l]/jaco[l]
        etay[l] = xpsi[l]/jaco[l]
        
        for a in range(4):
            Nx[a,l] = Npsi[a,l]*psix[l]+Neta[a,l]*etax[l]
            Ny[a,l] = Npsi[a,l]*psiy[l]+Neta[a,l]*etay[l]
        
        
    Ke = np.zeros((4,4))
    for a in range(4):
        for b in range(4):
            for l in range(nint):
                Ke[a,b] = Ke[a,b] + (Nx[a,l]*Nx[b,l]+Ny[a,l]*Ny[b,l])*jaco[l]*weights[l]
                
    Fe = np.zeros((4,))
    
    for a in range(4):
        for l in range(nint):
            Fe[a] = Fe[a] - 4*N[a,l]*jaco[l]*weights[l]
            
    return Ke, Fe

def Poisson(filepath):
    
    n_node, n_elem, N, E, C = read_input_file(filepath)
    
    K = np.zeros((n_node, n_node))
    F = np.zeros((n_node,))
    
    # create global stiffness matrix and force vector
    x = np.zeros((4,))
    y = np.zeros((4,))
    for i in range(n_elem):
        elem = E[i]
        nodes = []
        for j in range(4):
            nodes.append(int(elem[j+1])-1) # Add Python node number
            x[j] = N[nodes[-1],1]
            y[j] = N[nodes[-1],2]
        
        Ke, Fe = local_compute(x, y, 9)
        
        # assembly
        for j in range(4):
            for k in range(4):
                K[nodes[j], nodes[k]] += Ke[j,k]
            F[nodes[j]] += Fe[j]
#     F11 = F.copy()
    # impose boundary conditions
    for constrs in C:
        node_n = int(constrs[0])-1 # Python node number
        bc = constrs[2]
        f = K[node_n]
        F -= f*bc
        
        K[node_n,:] = 0
        K[:, node_n] = 0
        K[node_n, node_n] = 1
        F[node_n] = bc
    
    d = np.dot(inv(K), F)
    
    return d, n_node, n_elem, N
    

In [34]:
filepath = r'C:\Users\Yuquan Meng\Desktop\Mesh-Sixteen-Elements.m'
d,_,_,_ = Poisson(filepath)